In [12]:
##param_selection
'''
This cell includes the code to find the best parameters 
to build the final model
'''

import h2o
from h2o.estimators import H2ORandomForestEstimator
from h2o.estimators.deeplearning import H2ODeepLearningEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
import pandas as pd
h2o.init(max_mem_size= '16G', port = 54321)


train_path = 'data/dataset_2_train.csv'
df = pd.read_csv(train_path)

data = h2o.H2OFrame(df)
data['churned_within_30_days'] = data['churned_within_30_days'].asfactor()

training_columns =[u'f01', u'f02', u'f03', u'f04', u'f05', u'f06', u'f07',
       u'f08', u'f09', u'f10', u'f11', u'f12', u'f13', u'f14', u'f15', u'f16',
       u'f17', u'f18']
response_column = 'churned_within_30_days'
train, test = data.split_frame(ratios=[0.8])

m_GB = H2OGradientBoostingEstimator(ntrees=10, max_depth=9) 
m_GB.train(x=training_columns, y=response_column, training_frame=train, model_id= 'm_GB_10_9')
per_GB = m_GB.predict(test_data=test)

per_GB = m_GB.predict(test_data=test) 
out = pd.DataFrame(columns=['test','pred'])
out['test'] = test['churned_within_30_days'].as_data_frame()
out['pred'] = per_GB['predict'].as_data_frame()

FP =  float((out[(out['test']==1) & (out['pred'] == 0)]).shape[0])
TP =  float((out[(out['test']==0) & (out['pred'] == 0)]).shape[0])
FN =  float((out[(out['test']==0) & (out['pred'] == 1)]).shape[0])
TN =  float((out[(out['test']==1) & (out['pred'] == 1)]).shape[0])

FPR = FP /(FP + TP)
TPR = TP /(TP +FN)

print (FPR, TPR)

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.161-b12, mixed mode)
  Starting server from c:\Python27\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: c:\users\ebigdeli\appdata\local\temp\1\tmp94wp8s
  JVM stdout: c:\users\ebigdeli\appdata\local\temp\1\tmp94wp8s\h2o_ebigdeli_started_from_python.out
  JVM stderr: c:\users\ebigdeli\appdata\local\temp\1\tmp94wp8s\h2o_ebigdeli_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.18.0.5
H2O cluster version age:,2 days
H2O cluster name:,H2O_from_python_ebigdeli_5qtgmc
H2O cluster total nodes:,1
H2O cluster free memory:,14.22 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"


Parse progress: |█████████████████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%
(0.029696578437701744, 0.9929968287526427)


In [14]:
##final_model
'''
This cell builds the final model and precit the label
validation set.
'''
    
train_path = 'data/dataset_2_train.csv'
df = pd.read_csv(train_path)

train = h2o.H2OFrame(df)
train['churned_within_30_days'] = train['churned_within_30_days'].asfactor()

m_GB = H2OGradientBoostingEstimator(ntrees=10, max_depth=9, distribution = 'bernoulli') 
m_GB.train(x=training_columns, y=response_column, training_frame=train)

##Validation
validation_path = 'data/dataset_2_validation.csv'
Gradient_Boosting = pd.read_csv(validation_path)

validation = h2o.H2OFrame(Gradient_Boosting)
per_GB = m_GB.predict(test_data=validation)

output = per_GB.as_data_frame()
print output.shape
print (output[output['predict'] == 0]).shape
print (output[output['predict'] == 1]).shape

Gradient_Boosting['churned_within_30_days'] = output['predict']
Gradient_Boosting['p0'] = output['p0']
Gradient_Boosting['p1'] = output['p1']

out_path = 'data/Dataset_2_predict_xgboost.csv'
Gradient_Boosting.to_csv(out_path)

In [15]:
h2o.cluster().shutdown()

H2O session _sid_87d9 closed.
